# Introduction to Cloud Machine Learning with Flask API and CNTK



In [26]:
#load libraries
import os,sys
import pkg_resources
from flask import Flask, render_template, request, send_file
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import wget
import numpy as np
from PIL import Image, ImageOps
from urllib.request import urlretrieve
import requests
from cntk import load_model, combine
from io import BytesIO, StringIO
import base64
from IPython.core.display import display, HTML

print("System version: {}".format(sys.version))
print("Flask version: {}".format(pkg_resources.get_distribution("flask").version))
print("CNTK version: {}".format(pkg_resources.get_distribution("cntk").version))

System version: 3.5.2 |Continuum Analytics, Inc.| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Flask version: 0.10.1
CNTK version: 2.0.beta11.0


## Image classification with a pretrained CNTK model
The model is a ResNet with 18 layers, but there are other models that you can try.

In [ ]:
def maybe_download_model(filename='ResNet_18.model'):
    if(os.path.isfile(filename)):
        print("Model %s already downloaded" % filename)
    else:
        model_name_to_url = {
        'AlexNet.model':   'https://www.cntk.ai/Models/AlexNet/AlexNet.model',
        'AlexNetBS.model': 'https://www.cntk.ai/Models/AlexNet/AlexNetBS.model',
        'ResNet_18.model': 'https://www.cntk.ai/Models/ResNet/ResNet_18.model',
        'ResNet_152.model': 'https://migonzastorage.blob.core.windows.net/deep-learning/models/cntk/imagenet/ResNet_152.model'
        }
        url = model_name_to_url[filename] #TODO: try/except in case incorrect filename
        wget.download(url)

In [ ]:
model_name = 'ResNet_152.model'
maybe_download_model(model_name)

In [ ]:
def read_synsets(filename='synsets.txt'):
    with open(filename, 'r') as f:
        synsets = [l.rstrip() for l in f]
        labels = [" ".join(l.split(" ")[1:]) for l in synsets]
    return labels

labels = read_synsets()
print("Label length: ", len(labels))


Let's read images with PIL, plot them and crop them to the ImageNet size: `224x224`.

In [ ]:
def read_image_from_file(filename):
    img = Image.open(filename)
    return img
def read_image_from_ioreader(image_request):
    img = Image.open(BytesIO(image_request.read())).convert('RGB')
    return img
def read_image_from_request_base64(image_base64):
    img = Image.open(BytesIO(base64.b64decode(image_base64)))
    return img
def read_image_from_url(url):
    filename = urlretrieve(url)[0]
    return read_image_from_file(filename)

In [ ]:
def plot_image(img):
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [ ]:
imagepath = 'neko.jpg'
img = read_image_from_file(imagepath)
plot_image(img)

In [ ]:
imagefile = open(imagepath, 'rb')
print(type(imagefile))
img = read_image_from_ioreader(imagefile)
plot_image(img)

In [ ]:
imagefile = open(imagepath, 'rb')
image_base64 = base64.b64encode(imagefile.read())
print("String of %d characters" % len(image_base64))
img = read_image_from_request_base64(image_base64)
plot_image(img)

In [ ]:
imageurl = 'https://pbs.twimg.com/profile_images/269279233/llama270977_smiling_llama_400x400.jpg'
img = read_image_from_url(imageurl)
plot_image(img)

Once we have the image, the model file and the sysntets, the next step is to load the model and perform a prediction.

In [ ]:
%%time
z = load_model(model_name)

In [ ]:
def predict(model, image, labels, number_results = 5):
    #Crop and center the image
    img = ImageOps.fit(image, (224, 224), Image.ANTIALIAS)
    #Transform the image for CNTK format
    img = np.array(img, dtype=np.float32)
    img = np.ascontiguousarray(np.transpose(img, (2, 0, 1)))
    # Use last layer to make prediction
    z_out = combine([model.outputs[3].owner])
    result = np.squeeze(z_out.eval({z_out.arguments[0]:[img]}))
    # Sort probabilities 
    prob_idx = np.argsort(result)[::-1][:number_results]
    pred = [labels[i] for i in prob_idx]
    return pred
 

In [ ]:
resp = predict(z, img, labels, 5)
print(resp)
resp = predict(z, img2, labels, 5)
print(resp)
resp = predict(z, read_image_from_url('http://www.awf.org/sites/default/files/media/gallery/wildlife/Hippo/Hipp_joe.jpg'), labels, 5)
print(resp)

## Set up Flask API

In [17]:
headers = {'Content-type':'application/json'}
data = {"param":"1"}
try:
    res = requests.post('http://127.0.0.1:5000/api/v1/classify_image', data=json.dumps(data), headers=headers)
    print(res.content)
except Exception:
    print("ERROR: You should start the flask server!")
    print("To do it run 'python api.py' inside a cntk environment")

ERROR: You should start the flask server!
To do it run 'python api.py' inside a cntk environment


In [15]:
image_request = open(imagepath, 'rb')
files = {'image': image_request}

In [ ]:
%%time
res = requests.post('http://127.0.0.1:5000/api/v1/classify_image', files=files).json()
print(res['message'])

## Configure gunicorn

In this tutorial it is explained [how to set up gunicorn with flask and nigx](https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04).

Make sure that you install gunicorn inside the `cntk` environment: `pip install gunicorn`. The executable will be available at `/anaconda/envs/cntk/bin/gunicorn`.  

After creating the file [wsgi.py](./wsgi.py), you can start up gunicorn with:
```
/anaconda/envs/cntk/bin/gunicorn --bind 0.0.0.0:8008 wsgi:app
```
You have to get something like this:
```
[2017-03-18 15:40:54 +0000] [35940] [INFO] Starting gunicorn 19.7.0
[2017-03-18 15:40:54 +0000] [35940] [INFO] Listening at: http://0.0.0.0:8008 (35940)
[2017-03-18 15:40:54 +0000] [35940] [INFO] Using worker: sync
[2017-03-18 15:40:54 +0000] [35943] [INFO] Booting worker with pid: 35943
```
Once gunicorn is up, you can test that the endpoint is working. In your browser you can put `http://the-name-of-your-server:8008/`. You should see something like this. 

*NOTE: Make sure that you open the ports of your server. In case you are in an Azure machine, you have to go to your Network security group, then Inbound security rules and then add a new rule with the port 8008.* 


In [56]:
server_name = 'http://the-name-of-your-server'
server_name = 'http://migonzacpulinuxvm.southcentralus.cloudapp.azure.com'
port = 8008
complete_url = '{}:{}/'.format(server_name, port)
#print("Making a request to: {}".format(complete_url))
res = requests.get(complete_url)
display(HTML(res.text))

## Configure Nginx

You need to install nginx and some other libraries: `sudo apt-get install nginx upstart -y`

The first step is to modify `cntk_api.conf` and copy it to `/etc/init`.

```
# (2 Workers * CPU Cores) + 1
# ---------------------------
# For 1 core  -> (2*1)+1 = 3
# For 2 cores -> (2*2)+1 = 5
# For 4 cores -> (2*4)+1 = 9
```
You can specify the amount of workers by passing the argument --workers=[n].

In [60]:
%%bash 
sudo cp cntk_api.conf /etc/init

In [61]:
%%bash
sudo cp cntk_api /etc/nginx/sites-available/
sudo ln -s /etc/nginx/sites-available/cntk_api /etc/nginx/sites-enabled